In [1]:
import os
ROOT = os.path.join('..', '..')
import sys
sys.path.append(ROOT)
# 
import numpy as np

from src.models import UserUserColaborativeFiltering

# Train - Test Split

In [2]:
from src.utils import read_ml, ml_train_test_split

ml_complete_df, ml_users_df, ml_ratings_df, ml_movies_df, ml_genres_df = read_ml()

MAX_K = 15
ml_ratings_train_df, ml_ratings_test_df = ml_train_test_split(ml_ratings_df=ml_ratings_df, min_user_test_samples=MAX_K)

# User-User Colaborative Filtering

In [3]:
recommender = UserUserColaborativeFiltering(ml_movies_df, ml_users_df)
recommender.fit(ml_ratings_train_df)

In [4]:
metrics = recommender.evaluate(ml_ratings_test_df, MAX_K)

100%|██████████| 442/442 [00:12<00:00, 34.75it/s]


In [5]:
print("Average Precision@K:", round(np.mean(metrics['precision@k']), 4))
print("Average Recall@K:", round(np.mean(metrics['recall@k']), 4))
print("Average F1@K:", round(np.mean(metrics['f1@k']), 4))
print()
print("Best Precision@K:", round(np.max(metrics['precision@k']), 4))
print("Best Recall@K:", round(np.max(metrics['recall@k']), 4))
print("Best F1@K:", round(np.max(metrics['f1@k']), 4))

Average Precision@K: 0.1332
Average Recall@K: 0.0224
Average F1@K: 0.034

Best Precision@K: 0.8667
Best Recall@K: 0.1304
Best F1@K: 0.1579
